In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

from sklearn.cluster import SpectralClustering

import pycombo

import pandas as pd

torch.set_printoptions(sci_mode=False)

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
lr = 0.0001
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 1000
seed = 165

In [4]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [5]:
def community_detection(G,method):
    
    adj = nx.to_numpy_matrix(G)
    adj = np.expand_dims(adj, axis=0)
    
    #Combo
    combo_partition = pycombo.execute(G)
    nb_comm = max(combo_partition[0].values())+1
    combo_mod = combo_partition[1]

    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj))
    adj_norm = torch.FloatTensor(np.array(adj_norm))

    # features
    if method == "louvain":
        partition = community_louvain.best_partition(G)
        #get binary matrix of the partition
        nb_community = max(list(partition.values())) + 1
        communities =  np.array(list(partition.values())).reshape(-1)
        C = np.eye(nb_community)[communities]
    elif method == "spectral":
        global lr
        lr = 0.001
        partition = spectralPartition(adj,nb_comm)
        C = partition.float()
        nb_community = C.shape[1]
    
    features = torch.FloatTensor(C)
    features = features.unsqueeze(0)
    C = torch.FloatTensor(C).unsqueeze(0)

    Q = modularity_matrix(adj)
    
    # Model and optimizer

    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_community)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        Q = Q.cuda()
        C = C.cuda()

    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    

    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        C = model(features, adj_norm)
        loss = modularity(C,Q)

        loss.backward()

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            init_mod = loss
        else:
            if loss < best_loss:
                best_loss = loss

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return nb_community,-init_mod,-best_loss,combo_mod 

In [6]:
def spectralPartition(A,nb_comm):
    A = A[0] # TODO 3D
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [7]:
G = [nx.karate_club_graph(),nx.les_miserables_graph(),nx.florentine_families_graph(),nx.house_graph()]
methods = ["louvain","spectral"]
nb_communities = []
init_mods = []
finetuned_mods = []
combo_mods = []
methodss = []
networks = ["Karate Club","Karate Club","Les Miserables","Les Miserables","Florentine","Florentine","House Graph","House Graph"]
for network in G:
    for method in methods:
        nb_community,init_mod,finetuned_mod,combo_mod = community_detection(network,method)
        nb_communities.append(nb_community)
        init_mods.append(init_mod.cpu().detach().numpy()[0][0])
        finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
        combo_mods.append(combo_mod)
        methodss.append(method)

Epoch: 0001 Modularity: 0.41559827 time: 0.3467s
Epoch: 0101 Modularity: 0.41583103 time: 0.0000s
Epoch: 0201 Modularity: 0.41620481 time: 0.0000s
Epoch: 0301 Modularity: 0.41724196 time: 0.0000s
Epoch: 0401 Modularity: 0.41724196 time: 0.0000s
Epoch: 0501 Modularity: 0.41724196 time: 0.0000s
Epoch: 0601 Modularity: 0.41724196 time: 0.0000s
Epoch: 0701 Modularity: 0.41724196 time: 0.0000s
Epoch: 0801 Modularity: 0.41724196 time: 0.0000s
Epoch: 0901 Modularity: 0.41724196 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.3977s
Epoch: 0001 Modularity: 0.34015450 time: 0.0000s
Epoch: 0101 Modularity: 0.36307594 time: 0.0000s
Epoch: 0201 Modularity: 0.36978254 time: 0.0000s
Epoch: 0301 Modularity: 0.37422895 time: 0.0000s
Epoch: 0401 Modularity: 0.37868163 time: 0.0000s
Epoch: 0501 Modularity: 0.39825177 time: 0.0000s
Epoch: 0601 Modularity: 0.40203816 time: 0.0000s
Epoch: 0701 Modularity: 0.40203816 time: 0.0000s
Epoch: 0801 Modularity: 0.40203816 time: 0.0000s
Epoch: 0901 Modula

In [8]:
df = pd.DataFrame([networks,methodss,nb_communities,init_mods,finetuned_mods,combo_mods]).transpose()

In [9]:
df.columns = ["Network","Method","Communities","Initial Modularity","Finetuned Moduclarity","Combo Modularity"]

In [10]:
df

,Network,Method,Communities,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,0.415598,0.417242,0.419790
1,Karate Club,spectral,4,0.340154,0.402038,0.419790
2,Les Miserables,louvain,6,0.565416,0.565822,0.566688
3,Les Miserables,spectral,6,0.117456,0.388901,0.566688
4,Florentine,louvain,3,0.398750,0.398750,0.398750
5,Florentine,spectral,3,-0.171250,0.361250,0.398750
6,House Graph,louvain,2,0.111111,0.111111,0.111111
7,House Graph,spectral,2,-0.347222,0.111110,0.111111
